In [2]:
import numpy as np

class Transportation_Problem:
    def __init__(self,C,S,D):
        self.C = np.array(C).astype(float) # cost matrix
        self.S = np.array(S).astype(float) # supply matrix
        self.D = np.array(D).astype(float) # demand matrix
        self.A = np.zeros(self.C.shape).astype(float) # allocation matrix
        self.total_cost = 0
        self.no_alloc = 0
        self.total_demand = 0
        self.total_supply = 0
        self.rows,self.cols = self.C.shape
    
    def is_balanced(self):
        self.total_demand = np.sum(self.D)
        self.total_supply = np.sum(self.S)
        return self.total_demand == self.total_supply

    def make_balanced(self):
        if self.total_demand<self.total_supply:
            self.D = np.append(self.D,np.array([self.total_supply-self.total_demand]))
            self.C = np.append(self.C,np.zeros((self.rows,1)),axis=1)
            self.total_demand = np.sum(self.D)
            self.rows,self.cols = self.C.shape

        elif self.total_demand>self.total_supply:
            self.S = np.append(self.S,np.array([self.total_demand-self.total_supply]))
            self.C = np.append(self.C,np.zeros((1,self.cols)),axis=0)
            self.total_supply = np.sum(self.S)
            self.rows,self.cols = self.C.shape

    def is_degenerate(self):
        return (self.rows+self.cols-1) == self.no_alloc

    def make_non_degenerate(self):
        diff = (self.rows+self.cols-1) - self.no_alloc
        empty_cells = []
        for r in range(self.rows):
            for c in range(self.cols):
                if self.A[r][c]==0:
                    empty_cells.append((self.C[r][c],r,c))
        empty_cells.sort()
        for i in range(diff):
            _,r,c = empty_cells[i]
            self.A[r][c] = 0.001

    def vogel_approximation(self):
        def get_min_min2(M):
            m1,m2 = np.inf,np.inf
            for m in M:
                if m<=m1:
                    m1,m2 = m,m1
                elif m<=m2:
                    m2 = m
            return m1,m2
        
        def get_max_penalty(C):
            row_penalty = {}
            for r in range(C.shape[0]):
                m1,m2 = get_min_min2(C[r,:])
                if m1!=np.inf and m2!=np.inf:
                    row_penalty[r] = m2-m1
                elif m1!=np.inf:
                    row_penalty[r] = m1

            col_penalty = {}
            for c in range(C.shape[1]):
                m1,m2 = get_min_min2(C[:,c])
                if m1!=np.inf and m2!=np.inf:
                    col_penalty[c] = m2-m1
                elif m1!=np.inf:
                    col_penalty[c] = m1

            print("row penalty:"," ".join([str(row_penalty[i]) if i in
row_penalty else "-" for i in range(C.shape[0]) ]))
            print("col penalty:"," ".join([str(col_penalty[i]) if i in
col_penalty else "-" for i in range(C.shape[1]) ]))

            if not (row_penalty or col_penalty):
                return -1,-1,None

            r_mp,c_mp = max(row_penalty,key = lambda x:
                row_penalty[x]),max(col_penalty,key = lambda x: col_penalty[x])
            if r_mp>c_mp:
                i, mp = r_mp,row_penalty[r_mp]
                j = 0
                mn = C[i,j]
                for c in range(C.shape[1]):
                    if C[i,c]<mn:
                        j = c
                        mn = C[i,c]
            else:
                j,mp = c_mp,col_penalty[c_mp]
                i = 0
                mn = C[i,j]
                
                for r in range(C.shape[0]):
                    if C[r,j]<mn:
                        i = r
                        mn = C[r,j]
            
            return i,j,mp

        C,S,D = self.C.copy(),self.S.copy(),self.D.copy()
        self.A = np.zeros(self.C.shape).astype(float)
        r,c = self.C.shape
        self.total_cost = 0
        self.no_alloc = 0

        count = 1
        print(f"\nP{count}")
        count += 1
        i,j,max_penalty = get_max_penalty(C)
        while max_penalty!=None:
            print(f"row = S{i+1} col = D{j+1} max penalty = {max_penalty}")
            x = min(S[i],D[j])
            S[i] -= x
            D[j] -= x
            self.total_cost += x*C[i,j]
            self.no_alloc += 1
            self.A[i,j] = x
            if S[i]<=D[j]:
                for k in range(c):
                    C[i,k] = np.inf
            if S[i]>=D[j]:
                for k in range(r):
                    C[k,j] = np.inf
            print(f"\nP{count}")
            count += 1
            i,j,max_penalty = get_max_penalty(C)

def main():
    s = int(input("Enter number of supply:"))
    d = int(input("Enter number of demand:"))
    print("Enter the cost matrix:")
    C = [list(map(int,input().split())) for _ in range(s)]
    print("Enter the supply matrix:")
    S = list(map(int,input().split()))
    print("Enter the demand matrix:")
    D = list(map(int,input().split()))
    tp = Transportation_Problem(C,S,D)
    td,ts = sum(D),sum(S)
    print(f"total demand = {td},total supply = {ts}")

    if tp.is_balanced():
        print("Total demand is equal to total supply,")
        print("Hence, Balanced Transportation Problem.")
    else:
        print("Total demand is not equal to total supply,")
        print("Hence, Unbalanced Transportation Problem.")

        if td<ts:
            print("Making it balanced by adding dummy demand.")
        elif td>ts:
            print("Making it balanced by adding dummy supply.")
        print()
        tp.make_balanced()
    
    print("Cost Matrix:\n",tp.C)
    print("Supply Matrix:\n",tp.S)
    print("Demand Matrix:\n",tp.D)

    print("Initial basic feasible solution: VAM")

    tp.vogel_approximation()

    print()
    print("Allocations:\n",tp.A)
    print()
    print("Total Cost: ",tp.total_cost)
    print()
    print("No of Allocation: ",tp.no_alloc)
    print()
    print(f"m+n-1={tp.rows}+{tp.cols}-1 = {tp.rows+tp.cols-1}")
    if tp.is_degenerate() and tp.is_balanced():
        print("Since m+n-1 is equal to no. of allocations,\n Therefore it is not a Degenerate Solution")
    else:
        print("Since m+n-1 is not equal to no. of allocations,\n Therefore it is a Degenerate Solution")
    
        print("allotting dummy allocations ... ")
        tp.make_non_degenerate()
        print("\nNew Allocations:\n",tp.A)

if __name__=="__main__":
    main()

Enter number of supply:3
Enter number of demand:4
Enter the cost matrix:
6 4 1 5
8 9 2 7
4 3 6 2
Enter the supply matrix:
14 16 5
Enter the demand matrix:
6 10 10 4
total demand = 30,total supply = 35
Total demand is not equal to total supply,
Hence, Unbalanced Transportation Problem.
Making it balanced by adding dummy demand.

Cost Matrix:
 [[6. 4. 1. 5. 0.]
 [8. 9. 2. 7. 0.]
 [4. 3. 6. 2. 0.]]
Supply Matrix:
 [14. 16.  5.]
Demand Matrix:
 [ 6. 10. 10.  4.  5.]
Initial basic feasible solution: VAM

P1
row penalty: 1.0 2.0 2.0
col penalty: 2.0 1.0 1.0 3.0 0.0
row = S3 col = D4 max penalty = 3.0

P2
row penalty: 1.0 2.0 3.0
col penalty: 2.0 1.0 1.0 - 0.0
row = S3 col = D5 max penalty = 3.0

P3
row penalty: 1.0 2.0 -
col penalty: 2.0 5.0 1.0 - 0.0
row = S1 col = D2 max penalty = 5.0

P4
row penalty: 1.0 2.0 -
col penalty: 2.0 - 1.0 - 0.0
row = S2 col = D5 max penalty = 2.0

P5
row penalty: 5.0 6.0 -
col penalty: 2.0 - 1.0 - -
row = S2 col = D3 max penalty = 6.0

P6
row penalty: 6.0 8.0 -

In [1]:
import numpy as np

def get_balanced(supply, demand, costs, penalties = None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        return new_supply, demand, new_costs
    
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

def vogel(supply, demand,costs):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    m=len(supply)
    n=len(demand)
    i = 0
    j = 0
    bfs = []
    
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    cost=0
    bfs_arr = [[0 for i in range(n)] for j in range(m)] 
    
    for item in bfs:
        bfs_arr[item[0][0]][item[0][1]]=item[1]
    print('\n The initial bfs is:\n',bfs_arr)
    
    for item in bfs:
        cost=cost+costs[item[0][0]][item[0][1]]*item[1]
    print('total bfs cost is: ',cost)
    
    return bfs

def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs

def get_ws(bfs, costs, us, vs):
    ws = []
    
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    
    return ws_copy[-1][0]

def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

def transportation_method(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = get_balanced(supply, demand, costs)
    
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(vogel(balanced_supply, balanced_demand,costs))
    ans = np.zeros((len(costs), len(costs[0])))
    
    for (i, j), v in basic_variables:
        ans[i][j] = int(v)

    return ans

def get_total_cost(costs, ans):
    total_cost = 0
    
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            total_cost += cost * ans[i][j]
    return total_cost

m=int(input("Enter m:"))
n=int(input("Enter n:"))
print("Enter all costs in one line:")

entries = list(map(int, input().split()))  
costs = np.array(entries).reshape(m, n)

print('\n The costs are:\n',costs)
print('\n')

supply = list(map(int,input("Enter the supply values: ").strip().split()))[:m]
print('\n')

demand = list(map(int,input("Enter the demand values: ").strip().split()))[:n]
print('\n')

ans = transportation_method(supply, demand, costs)
print('\n')
print('\n The optimal solution is:\n',ans)
print('Total optimal cost: ', get_total_cost(costs, ans))

Enter m:4
Enter n:5
Enter all costs in one line:
4 3 1 2 6 5 2 3 4 5 3 5 6 3 2 2 4 4 5 3

 The costs are:
 [[4 3 1 2 6]
 [5 2 3 4 5]
 [3 5 6 3 2]
 [2 4 4 5 3]]


Enter the supply values: 80 60 40 20


Enter the demand values: 60 60 30 40 10



 The initial bfs is:
 [[60, 20, 0, 0, 0], [0, 40, 20, 0, 0], [0, 0, 10, 30, 0], [0, 0, 0, 10, 10]]
total bfs cost is:  670



 The optimal solution is:
 [[10.  0. 30. 40.  0.]
 [ 0. 60.  0.  0.  0.]
 [30.  0.  0.  0. 10.]
 [20.  0.  0.  0.  0.]]
Total optimal cost:  420.0
